# Task 4 – Archiving the PEP individual in Provincial Election 2079, Nepal

– Archive the PEP individual in Provincial Election 2079, Nepal
Refer the website (https://election.ekantipur.com/?lng=eng) and scrap country(i.e. Nepal),
election_name (i.e. Parliament &amp; Provincial Election 2079), province, district, constituency_no,
candidate_name, and received_votes using both elections 2079

### Data scraping using BeautifulSoup, Selenium and Python

In [1]:
# Importing necessary libraries
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,NoSuchElementException,StaleElementReferenceException,ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options

### Initializing driver and Loading the wab page

In [2]:
#Initialize WebDriver
s=Service("D:/pythonprogramme/selenium/chromedriver.exe")
driver=webdriver.Chrome(service=s)

# Lodding page
driver.get('https://election.ekantipur.com/?lng=eng')

### Data scraping

In [3]:
WebDriverWait(driver,120).until(
    EC.presence_of_element_located((By.XPATH,'//*[@id="election-type"]/option[2]'))
)

provincial=driver.find_element(By.XPATH,'//*[@id="election-type"]/option[2]')
provincial.click()

# function for skipped pages
def retry_skipped_pages(skipped_pages, max_retries=5):
    for page in skipped_pages:
        attempt = 0
        success = False
        while attempt < max_retries and not success:
            try:
                driver.get(page)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,'//*[@id="election-type"]/option[2]'))
                )
                
                provincial=driver.find_element(By.XPATH,'//*[@id="election-type"]/option[2]')
                provincial.click()
                
                while True:
                    try:
                        search_button = driver.find_element(By.XPATH, '//*[@id="search-result"]')
                        driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-result"]')))
                        driver.execute_script("arguments[0].click();", search_button)
                        break
                    except (ElementClickInterceptedException, StaleElementReferenceException):
                        continue
                
                # data scraping
                page_source=driver.page_source
                soup=BeautifulSoup(page_source)

                country_='Nepal'
                election_name_='Parliament & Provincial Election 2079'
                provincial_election_='Provincial Election'
                province_=soup.find_all('li',class_='breadcrumb-item')[1].text.strip()
                district_=soup.find_all('li',class_='breadcrumb-item')[2].text.strip()
                
                print("This is from skipped pages at last")
                
                for data in soup.find_all('div',class_='col-md-4 col-sm-6 chartClass mb-5'):
                    cons=data.find('h3',class_='text-center').text.strip().split()
                    constituency_=' '.join(cons)
                    name_=data.find('div',class_='nominee-name').text.strip()
                    party_=data.find('div',class_='candidate-party-name').text.strip()
                    votes_=data.find('span',class_='vote-count font-md p-0').text.strip()

                    candidates.append([country_,election_name_,provincial_election_,province_,district_,constituency_,name_,party_,votes_])
                    print(country_,election_name_,provincial_election_,province_,district_,constituency_,name_,party_,votes_)

                    success = True
                    
            except (TimeoutException, StaleElementReferenceException, NoSuchElementException):
                attempt += 1
                print(f"Retrying {page} ({attempt}/{max_retries})")

            if attempt >= max_retries:
                print(f"Failed to retrieve {page} after {max_retries} attempts.")


'''
We run province and district loop from indexing 1 cause first options are default string province and district except in case of sudhurpaschim pradesh,
In pradesh 7 there is not default option 'district' and first option is Bajura district and it skips due to the loop starting index is 1,
To counter and get the data of bajura we add the url of it into skipped_pages that gonna scraped at last
'''
candidates=[]
skipped_pages=["https://election.ekantipur.com/pradesh-7/district-bajura?lng=eng"]
pro_index=0
dist_index=0

# Province Selection
province_options=driver.find_elements(By.XPATH,'//*[@id="provinceDD"]/option')[1:]
while pro_index < len(province_options):
    try:
        WebDriverWait(driver,60).until(
            EC.presence_of_all_elements_located((By.XPATH,'//*[@id="provinceDD"]/option'))
        )
        
        province_options=driver.find_elements(By.XPATH,'//*[@id="provinceDD"]/option')[1:]
        
        print("\n")
        print("province:",province_options[pro_index].text)
        print("\n")
        
        province_options[pro_index].click()
        time.sleep(1)
        
        # District Selection
        district_options=driver.find_elements(By.XPATH,'//*[@id="districtDD"]/option')[1:]
        while dist_index < len(district_options):
            district_options=driver.find_elements(By.XPATH,'//*[@id="districtDD"]/option')[1:]
            
            print("distirct:",district_options[dist_index].text)
            district_options[dist_index].click()
            time.sleep(1)
            
            while True:
                    try:
                        search_button = driver.find_element(By.XPATH, '//*[@id="search-result"]')
                        driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-result"]')))
                        driver.execute_script("arguments[0].click();", search_button)
                        break
                    except (ElementClickInterceptedException, StaleElementReferenceException):
                        continue
            
            # data scraping
            page_source=driver.page_source
            soup=BeautifulSoup(page_source)
            
            country_='Nepal'
            election_name_='Parliament & Provincial Election 2079'
            provincial_election_='Provincial Election'
            province_=soup.find_all('li',class_='breadcrumb-item')[1].text.strip()
            district_=soup.find_all('li',class_='breadcrumb-item')[2].text.strip()
            
            for data in soup.find_all('div',class_='col-md-4 col-sm-6 chartClass mb-5'):
                cons=data.find('h3',class_='text-center').text.strip().split()
                constituency_=' '.join(cons)
                name_=data.find('div',class_='nominee-name').text.strip()
                party_=data.find('div',class_='candidate-party-name').text.strip()
                votes_=data.find('span',class_='vote-count font-md p-0').text.strip()
                
                candidates.append([country_,election_name_,provincial_election_,province_,district_,constituency_,name_,party_,votes_])
#                 print(country_,election_name_,provincial_election_,province_,district_,constituency_,name_,party_,votes_)

            dist_index += 1
            
        pro_index += 1
        dist_index = 0
        
    except (NoSuchElementException, AttributeError):
        skipped_pages.append(driver.current_url)
        print("This page have some issues and will be retry at last")
        print(driver.current_url)
        
        if dist_index < len(district_options):
            dist_index += 1
            
        else:
            pro_index += 1
            dist_index = 0
            

# Calling the retry function for skipped pages
retry_skipped_pages(skipped_pages)

driver.quit()



province: Pradesh 1


distirct: Taplejung
distirct: Panchthar
distirct: Illam
distirct: Jhapa
distirct: Morang
distirct: Sunsari
distirct: Dhankuta
distirct: Sankhuwasabha
distirct: Bhojpur
distirct: Terhathum
distirct: Okhaldhunga
distirct: Khotang
distirct: Solukhumbu
distirct: Udayapur


province: Madhesh Pradesh


distirct: Saptari
distirct: Siraha
distirct: Dhanusa
distirct: Mahottari
distirct: Sarlahi
distirct: Rauthat
distirct: Bara
distirct: Parsa


province: Bagmati Pradesh


distirct: Sindhuli
distirct: Ramechhap
distirct: Dolkha
distirct: Sindhupalchowk
distirct: Kavrepalanchowk
distirct: Lalitpur
distirct: Bhaktapur
distirct: Kathmandu
distirct: Nuwakot
distirct: Rasuwa
distirct: Dhading
distirct: Makwanpur
distirct: Chitwan


province: Gandaki Pradesh


distirct: Gorkha
distirct: Lamjung
distirct: Tanahun
distirct: Syangja
distirct: Kaski
distirct: Manang
distirct: Mustang
distirct: Parbat
distirct: Myagdi
distirct: Baglung
distirct: Nawalparasi East


province: Lumbini 

### Creating DataFrame from scraped data

In [4]:
df=pd.DataFrame(candidates,columns=['country','election name','provincial election','province','district','constituency no','name','party name','received votes'])

Just printing DataFrame

In [5]:
df

,country,election name,provincial election,province,district,constituency no,name,party name,received votes
0,Nepal,Parliament & Provincial Election 2079,Provincial Election,Pradesh 1,Taplejung,Constituency : 1 (1),Til Kumar Menwang,CPN UML,"12,772"
1,Nepal,Parliament & Provincial Election 2079,Provincial Election,Pradesh 1,Taplejung,Constituency : 1 (2),Khagen Singh Hangam,Nepali Congress,"10,496"
2,Nepal,Parliament & Provincial Election 2079,Provincial Election,Pradesh 1,Panchthar,Constituency : 1 (1),Kamal Prasad Jabegu,CPN (Unified Socialist),"16,849"
3,Nepal,Parliament & Provincial Election 2079,Provincial Election,Pradesh 1,Panchthar,Constituency : 1 (2),Indra Bahadur Angwo,CPN Maoist,"15,685"
4,Nepal,Parliament & Provincial Election 2079,Provincial Election,Pradesh 1,Illam,Constituency : 1 (1),Khinu Langwa (Limbu),CPN (Unified Socialist),"13,371"
...,...,...,...,...,...,...,...,...,...
326,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Baitadi,Constituency : 1 (2),Shiva Raj Bhatta,Nepali Congress,"17,153"
327,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Darchula,Constituency : 1 (1),Bikram Singh Dhami,Nepali Congress,"15,127"
328,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Darchula,Constituency : 1 (2),Man Bahadur Dhami,CPN Maoist,"13,171"
329,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajura,Constituency : 1 (1),Naresh Kumar Shahi,CPN (Unified Socialist),"10,973"


### Droping duplicate rows if present

In [6]:
df.drop_duplicates(inplace=True)

Total no of provincial constituency = 330

But, we have 331 constituency in total, it is because in Bajhang district, Sudurpaschim Pradesh	
The winner of provincial election 2079, Prithivi Bahadur Singh died in car accident and re-election was re-conducted for the vacant post in 2081 BS.
Daman Bahadur Bhandari	wins the election. That's why there is 331 rows instead of 330.

Referencing >> https://en.wikipedia.org/wiki/Bajhang_1_(constituency)

In [7]:
df.shape[0]

331

We can see total constituency no of Bajhang is 3 instead of 2 

In [8]:
df[df['district']=='Bajhang']

,country,election name,provincial election,province,district,constituency no,name,party name,received votes
298,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,Constituency : 1 (1),Daman Bahadur Bhandari,CPN-UML,"11,613"
299,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,Constituency : 1 (1),Prithivi Bahadur Singh,Nepali Congress,"17,112"
300,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,Constituency : 1 (2),Bhim Bahadur Bhandari,CPN (Maoist Center),"13,879"


Just trying to separate the re-election from 2079 election

In [11]:
df.loc[298,'constituency no']='Constituency : 1 (1), By re-election 2081'

In [12]:
df[df['district']=='Bajhang']

,country,election name,provincial election,province,district,constituency no,name,party name,received votes
298,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,"Constituency : 1 (1), By re-election 2081",Daman Bahadur Bhandari,CPN-UML,"11,613"
299,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,Constituency : 1 (1),Prithivi Bahadur Singh,Nepali Congress,"17,112"
300,Nepal,Parliament & Provincial Election 2079,Provincial Election,Sudurpaschim Pradesh,Bajhang,Constituency : 1 (2),Bhim Bahadur Bhandari,CPN (Maoist Center),"13,879"


### Storing dataframe into csv file

In [14]:
df.to_csv('PEP individual in Provincial Election 2079, Nepal.csv')

Basics information of dataframe

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              331 non-null    object
 1   election name        331 non-null    object
 2   provincial election  331 non-null    object
 3   province             331 non-null    object
 4   district             331 non-null    object
 5   constituency no      331 non-null    object
 6   name                 331 non-null    object
 7   party name           331 non-null    object
 8   received votes       331 non-null    object
dtypes: object(9)
memory usage: 23.4+ KB


### Converting received votes column into integer

In [17]:
# Replace non-numeric values with NaN
df['received votes'] = pd.to_numeric(df['received votes'].astype(str).str.replace(',', ''), errors='coerce')

# Replace NaN with 0
df['received votes'] = df['received votes'].fillna(0).astype(int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   country              331 non-null    object
 1   election name        331 non-null    object
 2   provincial election  331 non-null    object
 3   province             331 non-null    object
 4   district             331 non-null    object
 5   constituency no      331 non-null    object
 6   name                 331 non-null    object
 7   party name           331 non-null    object
 8   received votes       331 non-null    int32 
dtypes: int32(1), object(8)
memory usage: 22.1+ KB


# Storing dataframe into database

### Importing necessary libraries

In [19]:
import pandas as pd
import MySQLdb

### Connecting database

In [20]:
conn=MySQLdb.connect(
    host='localhost',
    user='root',
    password='root',
    database='task'
)
cursor=conn.cursor()

### Storing data into database

In [23]:
# Iterate over DataFrame rows and execute insert query
for i, row in df.iterrows():
    cursor.execute("""
    INSERT INTO task4(
    country,
    `election name`,
    `provincial election`,
    province,
    district,
    `constituency no`,
    name,
    `party name`,
    `received votes`
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['country'],
        row['election name'],
        row['provincial election'],
        row['province'],
        row['district'],
        row['constituency no'],
        row['name'],
        row['party name'],
        row['received votes']
    ))

# Committing the transaction
conn.commit()

# Closing the cursor and connection
cursor.close()
conn.close()
